# Initial Setup:

## Import Commands:

In [1]:
# Data Parsing:
import numpy as np
import pandas as pd

# Visualizations:
import seaborn as sns
import matplotlib.pyplot as plt

# Imputation:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

# Misc.
import warnings

### Options/Config:

In [2]:
# Disable warnings:
warnings.filterwarnings('ignore')

# Visualization options:
sns.set_theme()
pd.set_option('display.max_columns', None)
ppt_figsize = (18, 4)

# Load/Clean Input Data:

### Clean aircraft.csv:

In [3]:
# Load aviation data:
aircraft_df = pd.read_csv('./datasets/aircraft.csv')

def aircraft_cleaner(input_df):

    # Identify columns to keep:
    cols_to_keep = [
    'ev_id',
    'Aircraft_Key',
    'regis_no',
    'ntsb_no',
    'dest_apt_id',
    'dprt_apt_id',
    'damage',
    'cert_max_gr_wt',
    'acft_category',
    'homebuilt',
    'total_seats',
    'num_eng',
    'fixed_retractable',
    'type_last_insp',
    'afm_hrs_last_insp',
    'date_last_insp',
    'afm_hrs',
    'rwy_len',
    'rwy_width',
    'rwy_num',
    ]

    # Load encodings:
    encodings_df = pd.read_csv('./datasets/encodings.csv')
    
    # Convert 'date_last_insp' to datetime:
    input_df['date_last_insp'] = pd.to_datetime(input_df['date_last_insp'], errors='coerce')

    # Fill missing damages:
    input_df['damage'] = input_df['damage'].fillna('NONE')
    display(input_df['damage'].value_counts())

    # Create encoding mappings from encodings.csv:
    encoding_maps = {}
    for feature in encodings_df['Feature'].unique():
        feature_subset = encodings_df[encodings_df['Feature'] == feature]
        encoding_maps[feature] = dict(zip(feature_subset['Key'], feature_subset['Encoding']))

    # Map 'damage':
    input_df['damage'] = input_df['damage'].map(encoding_maps.get('damage', {}))

    # Binarize 'homebuilt' (Yes = 1, No = 0):
    input_df['homebuilt'] = input_df['homebuilt'].map({'Yes': 1, 'No': 0}).fillna(0)

    # Binarize 'fixed_retractable' (Yes = 1, No = 0):
    input_df['fixed_retractable'] = input_df['fixed_retractable'].map({'Yes': 1, 'No': 0}).fillna(1)

    # Map 'type_last_insp':
    input_df['type_last_insp'] = input_df['type_last_insp'].map(encoding_maps.get('type_last_insp', {}))

    # Only output relevant columns:
    input_df = input_df[cols_to_keep].drop_duplicates()

    # Remove any whitespace (sometimes causes merge issues):
    input_df = input_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Return cleaned DataFrame:
    return input_df

# Apply cleaning function:
aircraft_df = aircraft_cleaner(aircraft_df)

# Display outputs:
display(aircraft_df)
display(aircraft_df['damage'].value_counts())

damage
SUBS    22520
DEST     3233
NONE     1896
MINR     1150
UNK       188
Name: count, dtype: int64

,ev_id,Aircraft_Key,regis_no,ntsb_no,dest_apt_id,dprt_apt_id,damage,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,fixed_retractable,type_last_insp,afm_hrs_last_insp,date_last_insp,afm_hrs,rwy_len,rwy_width,rwy_num
0,20080211X00175,1,N530NA,DFW08RA039,NaN,NaN,3.0,NaN,HELI,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,20080107X00026,1,N8037W,SEA08LA057A,OQ3,O69,2.0,2400.0,AIR,0.0,4.0,1.0,1.0,3.0,75.0,2007-04-01,2865.0,2700.0,45.0,7
2,20080107X00026,2,N15EX,SEA08LA057B,O70,OQ3,3.0,1960.0,AIR,0.0,2.0,1.0,1.0,4.0,NaN,2007-01-01,340.0,2700.0,45.0,7
3,20080109X00036,1,N579RM,DFW08CA054,AXH,T20,3.0,2850.0,AIR,0.0,2.0,1.0,1.0,3.0,NaN,2007-09-01,516.0,5003.0,100.0,090
4,20080107X00027,1,N398J,DFW08LA055,PWA,PHX,1.0,9920.0,AIR,0.0,9.0,1.0,1.0,0.0,NaN,NaT,1871.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28982,20250225199761,1,N942AT,DCA25LA139,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN
28983,20250226199769,1,N43DN,ERA25LA130,NaN,MTN,1.0,NaN,AIR,0.0,4.0,1.0,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN
28984,20250227199772,1,N955NN,OPS25LA025A,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN
28985,20250227199772,2,N322RC,OPS25LA025B,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN


damage
3.0    22520
4.0     3233
1.0     1896
2.0     1150
0.0      188
Name: count, dtype: int64

### Clean Airport Data:

In [4]:
# Load aviation data:
DOT_airport_df = pd.read_csv('./datasets/airports.csv')

def airport_cleaner(input_df):

    # Identify columns to keep:
    cols_to_keep = [
    'Loc_Id',    
    'Owner_Type',
    'Fac_Use',
    'Dist_City_',
    'Elevation',
    'Region_Cod',
    'Icao_Ident',
    'Land_Area_',
    ]

    # Binarize 'Fac_Use' (PU = 1, PR = 0):
    input_df['Fac_Use'] = input_df['Fac_Use'].map({'PU': 1, 'PR': 0})

    # Binarize 'Owner_Type' (PU = 1, PR = 0):
    input_df['Owner_Type'] = input_df['Owner_Type'].map({'PU': 1, 'PR': 0})

    # If 'Owner_Type' is unknown, use value for 'Fac_Use':
    input_df['Owner_Type'] = input_df['Owner_Type'].fillna(input_df['Fac_Use'])

    # If unknown, assume inside of city:
    input_df['Dist_City_'] = input_df['Dist_City_'].fillna(0)

    # Only output relevant columns:
    input_df = input_df[cols_to_keep].drop_duplicates()

    # Remove any whitespace (sometimes causes merge issues):
    input_df = input_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Return cleaned DataFrame:
    return input_df

# Apply cleaning function:
DOT_airport_df = airport_cleaner(DOT_airport_df)

# Display outputs:
display(DOT_airport_df)
display(DOT_airport_df.isna().sum())

,Loc_Id,Owner_Type,Fac_Use,Dist_City_,Elevation,Region_Cod,Icao_Ident,Land_Area_
0,MYAM,1.0,1,3.0,8.0,NaN,MYAM,NaN
1,CYQQ,0.0,0,3.0,84.0,NaN,CYQQ,NaN
2,CYQG,1.0,1,0.0,622.0,NaN,CYQG,NaN
3,CO60,0.0,0,5.0,4800.0,ANM,NaN,NaN
4,8ND5,0.0,0,3.0,1020.0,AGL,NaN,NaN
...,...,...,...,...,...,...,...,...
19845,WV43,0.0,0,3.0,623.0,AEA,NaN,57.0
19846,2WV3,0.0,0,4.0,650.0,AEA,NaN,5.0
19847,MYER,1.0,1,2.0,16.0,NaN,MYER,NaN
19848,44WI,0.0,0,4.0,1298.0,AGL,NaN,NaN


Loc_Id            0
Owner_Type        0
Fac_Use           0
Dist_City_        0
Elevation         0
Region_Cod      146
Icao_Ident    17155
Land_Area_    10149
dtype: int64

In [5]:
# Load aviation data:
NTAD_airport_df = pd.read_csv('./datasets/NTAD_Aviation_Facilities.csv')

def airport_cleaner(input_df):

    # Identify columns to keep:
    cols_to_keep = [
    'ARPT_ID',    
    'OWNERSHIP_TYPE_CODE',
    'FACILITY_USE_CODE',
    'DIST_CITY_TO_AIRPORT',
    'ELEV',
    'REGION_CODE',
    'ICAO_ID',
    'ACREAGE',
    ]

    # Binarize 'FACILITY_USE_CODE' (PU = 1, PR = 0):
    input_df['FACILITY_USE_CODE'] = input_df['FACILITY_USE_CODE'].map({'PU': 1, 'PR': 0})

    # Binarize 'OWNERSHIP_TYPE_CODE' (PU = 1, PR = 0):
    input_df['OWNERSHIP_TYPE_CODE'] = input_df['OWNERSHIP_TYPE_CODE'].map({'PU': 1, 'PR': 0})

    # If 'OWNERSHIP_TYPE_CODE' is unknown, use value for 'FACILITY_USE_CODE':
    input_df['OWNERSHIP_TYPE_CODE'] = input_df['OWNERSHIP_TYPE_CODE'].fillna(input_df['FACILITY_USE_CODE'])

    # If unknown, assume inside of city:
    input_df['DIST_CITY_TO_AIRPORT'] = input_df['DIST_CITY_TO_AIRPORT'].fillna(0)

    # Only output relevant columns:
    input_df = input_df[cols_to_keep].drop_duplicates()

    # Remove any whitespace (sometimes causes merge issues):
    input_df = input_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Return cleaned DataFrame:
    return input_df

# Apply cleaning function:
NTAD_airport_df = airport_cleaner(NTAD_airport_df)

# Display outputs:
display(NTAD_airport_df)
display(NTAD_airport_df.isna().sum())

,ARPT_ID,OWNERSHIP_TYPE_CODE,FACILITY_USE_CODE,DIST_CITY_TO_AIRPORT,ELEV,REGION_CODE,ICAO_ID,ACREAGE
0,0J0,1.0,1,3.0,463.1,ASO,NaN,36.0
1,2A8,1.0,1,1.0,786.0,ASO,NaN,37.0
2,AL03,0.0,0,3.0,369.0,ASO,NaN,NaN
3,EET,1.0,1,4.0,585.6,ASO,KEET,125.0
4,AL39,0.0,0,0.0,510.0,ASO,NaN,NaN
...,...,...,...,...,...,...,...,...
20174,MBAC,0.0,0,0.0,9.0,NaN,MBAC,NaN
20175,MBGT,1.0,1,2.0,11.0,NaN,MBGT,NaN
20176,MBNC,1.0,1,0.0,12.0,NaN,MBNC,NaN
20177,MBPV,1.0,1,1.0,13.0,NaN,MBPV,NaN


ARPT_ID                     0
OWNERSHIP_TYPE_CODE         0
FACILITY_USE_CODE           0
DIST_CITY_TO_AIRPORT        0
ELEV                        0
REGION_CODE               147
ICAO_ID                 17477
ACREAGE                 10416
dtype: int64

#### Merge Airport DataFrames:

In [6]:
# Standardize column names:
NTAD_airport_df = NTAD_airport_df.rename(columns={
    'ARPT_ID': 'Loc_Id',
    'OWNERSHIP_TYPE_CODE': 'Owner_Type',
    'FACILITY_USE_CODE': 'Fac_Use',
    'DIST_CITY_TO_AIRPORT': 'Dist_City_',
    'ELEV': 'Elevation',
    'REGION_CODE': 'Region_Cod',
    'ICAO_ID': 'Icao_Ident',
    'ACREAGE': 'Land_Area_',
})

# Merge both datasets on 'Loc_Id':
airport_df = pd.merge(NTAD_airport_df, DOT_airport_df, on='Loc_Id', how='outer', suffixes=('_NTAD', '_DOT'))

# Select only six final columns, prioritizing non-null values where overlap exists:
airport_df['Owner_Type'] = airport_df['Owner_Type_NTAD'].combine_first(airport_df['Owner_Type_DOT'])
airport_df['Fac_Use'] = airport_df['Fac_Use_NTAD'].combine_first(airport_df['Fac_Use_DOT'])
airport_df['Dist_City_'] = airport_df['Dist_City__NTAD'].combine_first(airport_df['Dist_City__DOT'])
airport_df['Elevation'] = airport_df['Elevation_NTAD'].combine_first(airport_df['Elevation_DOT'])
airport_df['Region_Cod'] = airport_df['Region_Cod_NTAD'].combine_first(airport_df['Region_Cod_DOT'])
airport_df['Icao_Ident'] = airport_df['Icao_Ident_NTAD'].combine_first(airport_df['Icao_Ident_DOT'])
airport_df['Land_Area_'] = airport_df['Land_Area__NTAD'].combine_first(airport_df['Land_Area__DOT'])

# Ensure NA values are detected:
airport_df['Elevation'] = airport_df['Elevation'].replace(['', ' '], pd.NA)

# Create duplicate rows where 'Icao_Ident' is non-null and replace 'Loc_Id' with 'Icao_Ident'
icao_rows = airport_df[airport_df['Icao_Ident'].notna()].copy()
icao_rows['Loc_Id'] = icao_rows['Icao_Ident']

# Append duplicated rows back to the DataFrame
airport_df = pd.concat([airport_df, icao_rows], ignore_index=True)

# Keep only the required columns:
airport_df = airport_df[['Loc_Id', 'Owner_Type', 'Fac_Use', 'Dist_City_', 'Elevation', 'Region_Cod', 'Land_Area_']].reset_index(drop=True)

### Clean Flight_Crew.csv

In [7]:
# Load aviation data:
flightcrew_df = pd.read_csv('./datasets/Flight_Crew.csv')

def flightcrew_cleaner(input_df):
    # Identify columns to keep:
    cols_to_keep = [
        'ev_id',
        'Aircraft_Key',
        'crew_no',
        'crew_age',
        'med_certf',
    ]

    # Load encodings:
    encodings_df = pd.read_csv('./datasets/encodings.csv')

    # Create encoding mappings from encodings.csv:
    encoding_maps = {}
    for feature in encodings_df['Feature'].unique():
        feature_subset = encodings_df[encodings_df['Feature'] == feature]
        encoding_maps[feature] = dict(zip(feature_subset['Key'], feature_subset['Encoding']))

    # Map 'med_certf' to numerical encoding:
    input_df['med_certf'] = input_df['med_certf'].fillna('UNK')
    input_df['med_certf'] = input_df['med_certf'].map(encoding_maps.get('med_certf', {}))

    # Groupby 'ev_id' and 'Aircraft_Key' and get the average age of the crew:
    input_df['crew_age'] = input_df.groupby(['ev_id', 'Aircraft_Key'])['crew_age'].transform('mean')

    # Get the max value of 'crew_no' for each 'ev_id'+'Aircraft_Key' pairing:
    input_df['crew_no'] = input_df.groupby(['ev_id', 'Aircraft_Key'])['crew_no'].transform('max')

    # Use KNN Imputer to fill missing 'crew_age' based on 'crew_no' and 'med_certf':
    knn_features = ['crew_no', 'med_certf', 'crew_age']
    knn_df = input_df[knn_features].copy()
    knn_df = knn_df.astype(float, errors='ignore')

    # Instantiate/apply imputer:
    imputer = KNNImputer(n_neighbors=5)
    knn_df_imputed = imputer.fit_transform(knn_df)

    # Put imputed values back into 'crew_age':
    knn_df_imputed = pd.DataFrame(knn_df_imputed, columns=knn_features, index=knn_df.index)
    input_df['crew_age'] = knn_df_imputed['crew_age']

    # Only output relevant columns:
    input_df = input_df[cols_to_keep].drop_duplicates()

    # Remove any whitespace (sometimes causes merge issues):
    input_df = input_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Return cleaned DataFrame:
    return input_df

# Apply cleaning function:
flightcrew_df = flightcrew_cleaner(flightcrew_df)

# Display outputs:
display(flightcrew_df)
display(flightcrew_df.isna().sum())

,ev_id,Aircraft_Key,crew_no,crew_age,med_certf
0,20080107X00026,1,1,56.0,3.0
1,20080107X00026,2,1,70.0,3.0
2,20080109X00036,1,1,54.0,3.0
3,20080107X00027,1,1,59.0,2.0
4,20080115X00051,1,1,36.0,3.0
...,...,...,...,...,...
29569,20250116199556,1,1,69.0,5.0
29570,20250121199567,1,1,72.0,3.0
29571,20250121199574,1,2,47.5,1.0
29572,20250121199574,1,2,47.5,6.0


ev_id           0
Aircraft_Key    0
crew_no         0
crew_age        0
med_certf       0
dtype: int64

### Clean events.csv:

In [8]:
# Load aviation data:
events_df = pd.read_csv('./datasets/events.csv')

def events_cleaner(input_df):
    # Identify columns to keep
    cols_to_keep = [
        'ev_id',
        'ev_date',
        'ev_time',
        'ev_year',
        'ev_month',
        'ev_nr_apt_loc',
        'ev_nr_apt_id',
        'apt_dist',
        'apt_elev',
        'light_cond',
        'sky_cond_ceil',
        'sky_cond_nonceil',
        'sky_ceil_ht',
        'sky_nonceil_ht',
        'vis_sm',
        'wx_temp',
        'wx_dew_pt',
        'wind_dir_deg',
        'wind_vel_kts',
        'gust_kts',
        'altimeter',
        'ev_highest_injury',
        'wx_cond_basic',
    ]

    # Load encodings
    encodings_df = pd.read_csv('./datasets/encodings.csv')

    # Convert 'ev_date' to datetime
    input_df['ev_date'] = pd.to_datetime(input_df['ev_date'], errors='coerce')

    # Convert 'ev_time' from HHMM to decimal (so 2230 becomes 22.5)
    def hhmm_to_decimal(t):
        # Handle missing/invalid values
        if pd.isna(t):
            return np.nan
        try:
            # Convert to string, remove decimals if any, then zero-fill to length 4
            s = str(int(t)).zfill(4)
            hh, mm = int(s[:2]), int(s[2:])
            output_time = round(hh + (mm / 60), 2)
            return output_time
        except:
            return np.nan

    # Apply the conversion function
    input_df['ev_time'] = input_df['ev_time'].apply(hhmm_to_decimal)

    # Fill missing sky condition data
    input_df['sky_cond_ceil'] = input_df['sky_cond_ceil'].fillna('UNK')
    input_df['sky_cond_nonceil'] = input_df['sky_cond_nonceil'].fillna('UNK')

    # Create encoding mappings from encodings.csv
    encoding_maps = {}
    for feature in encodings_df['Feature'].unique():
        feature_subset = encodings_df[encodings_df['Feature'] == feature]
        encoding_maps[feature] = dict(zip(feature_subset['Key'], feature_subset['Encoding']))

    # Map categorical variables to numerical encodings
    mapping_features = [
        'ev_nr_apt_loc', 'light_cond', 'sky_cond_nonceil',
        'sky_cond_ceil', 'ev_highest_injury', 'wx_cond_basic'
    ]
    for feature in mapping_features:
        if feature in input_df.columns and feature in encoding_maps:
            input_df[feature] = input_df[feature].map(encoding_maps[feature])

    # Convert 'wx_temp' and 'wx_dew_pt' from Celsius to Fahrenheit
    input_df['wx_temp'] = input_df['wx_temp'] * 9/5 + 32
    input_df['wx_dew_pt'] = input_df['wx_dew_pt'] * 9/5 + 32

    # KNN Imputer:
    numeric_impute_cols = ['sky_ceil_ht', 'sky_nonceil_ht']
    
    # Identify sky condition columns:
    knn_features = ['sky_cond_ceil', 'sky_ceil_ht', 'sky_cond_nonceil', 'sky_nonceil_ht']

    # Create a sub-DataFrame for KNNImputer
    df_knn = input_df[knn_features].copy()

    # KNNImputer requires float data, so ensure correct type:
    df_knn = df_knn.astype(float, errors='ignore')

    # Fit the imputer
    imputer = KNNImputer(n_neighbors=5)
    df_knn_imputed = imputer.fit_transform(df_knn)

    # Put imputed values back
    df_knn_imputed = pd.DataFrame(df_knn_imputed, columns=knn_features, index=df_knn.index)
    for col in numeric_impute_cols:
        input_df[col] = df_knn_imputed[col]

    injury_cols = ['inj_tot_f', 'inj_tot_m', 'inj_tot_n', 'inj_tot_s']
    # Make sure to fill missing injury counts with 0 for model input:
    for c in injury_cols:
        input_df[c] = input_df[c].fillna(0)

    # Create a mask for rows where ev_highest_injury is missing
    mask_missing_injury = input_df['ev_highest_injury'].isna()

    # Train set: rows with known ev_highest_injury
    train_df = input_df[~mask_missing_injury].copy()
    # Test set: rows with missing ev_highest_injury
    test_df = input_df[mask_missing_injury].copy()

    if not train_df.empty and not test_df.empty:
        # Prepare X/y for training
        X_train = train_df[injury_cols]
        y_train = train_df['ev_highest_injury']

        # Prepare X for prediction
        X_test = test_df[injury_cols]

        # Build and fit classifier
        clf = KNeighborsClassifier(n_neighbors=5)
        clf.fit(X_train, y_train)

        # Predict for missing
        input_df.loc[mask_missing_injury, 'ev_highest_injury'] = clf.predict(X_test)

    # Only output relevant columns
    input_df = input_df[cols_to_keep].drop_duplicates()

    # Remove any whitespace (sometimes causes merge issues)
    input_df = input_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Return cleaned DataFrame
    return input_df

# Apply cleaning function:
events_df = events_cleaner(events_df)

# Display outputs:
display(events_df)
display(events_df['ev_highest_injury'].value_counts())

,ev_id,ev_date,ev_time,ev_year,ev_month,ev_nr_apt_loc,ev_nr_apt_id,apt_dist,apt_elev,light_cond,sky_cond_ceil,sky_cond_nonceil,sky_ceil_ht,sky_nonceil_ht,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_vel_kts,gust_kts,altimeter,ev_highest_injury,wx_cond_basic
0,20080211X00175,2008-01-10,19.12,2008,1,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,32.0,32.0,0.0,NaN,0.0,0.000000,1.0,NaN
1,20080107X00026,2008-01-01,22.50,2008,1,1.0,OQ3,0.0,10.0,2.0,2.0,2.0,0.0,0.0,10.0,134.6,89.6,80.0,10.0,0.0,30.190001,1.0,2.0
2,20080109X00036,2008-01-01,22.00,2008,1,1.0,AXH,0.0,68.0,2.0,2.0,2.0,0.0,0.0,10.0,131.0,57.2,360.0,15.0,20.0,30.620001,1.0,2.0
3,20080107X00027,2008-01-03,8.42,2008,1,1.0,PWA,0.0,1299.0,6.0,2.0,2.0,0.0,0.0,10.0,32.0,32.0,170.0,16.0,0.0,30.639999,4.0,2.0
4,20080115X00051,2008-01-02,22.50,2008,1,1.0,FNL,1.0,5016.0,2.0,2.0,2.0,0.0,0.0,10.0,66.2,44.6,150.0,5.0,0.0,30.260000,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28510,20250224199755,2025-02-23,20.25,2025,2,NaN,NaN,NaN,NaN,2.0,1.0,0.0,4820.0,140.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,1.0,2.0
28511,20250225199761,2025-02-24,13.80,2025,2,NaN,NaN,NaN,NaN,2.0,0.0,3.0,0.0,3760.0,NaN,NaN,NaN,320.0,5.0,0.0,NaN,2.0,2.0
28512,20250226199769,2025-02-26,13.50,2025,2,0.0,NaN,0.0,NaN,2.0,4.0,0.0,1100.0,0.0,7.0,98.6,93.2,0.0,NaN,0.0,29.799999,3.0,1.0
28513,20250227199772,2025-02-26,20.63,2025,2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,1.0,2.0


ev_highest_injury
1.0    15720
4.0     5890
2.0     3790
3.0     3115
Name: count, dtype: int64

## Target Variable Identification:

### Clean Findings.csv

In [9]:
# Load aviation data:
findings_df = pd.read_csv('./datasets/Findings.csv')

def findings_cleaner(input_df):
    # Identify columns to keep
    cols_to_keep = [
        'ev_id',
        'Aircraft_Key',
        'category_no'
    ]

    # Dictionary mapping numeric codes to category labels
    category_dict = {
        1: 'Aircraft',
        2: 'Personnel issues',
        3: 'Environmental issues',
        4: 'Organizational issues',
        5: 'Not determined',
    }

    # Keep only relevant columns
    input_df = input_df[cols_to_keep].copy()

    # Map 'category_no' to category labels (ordinal encoding)
    input_df['category_no'] = input_df['category_no'].map(category_dict)
    input_df['category_no'] = input_df['category_no'].fillna('Not determined')

    # Create columns for each of the 5 categories (pseudo-OHE)
    cat_map = {
        'Aircraft': 'cat_Aircraft',
        'Personnel issues': 'cat_Personnel',
        'Environmental issues': 'cat_Environmental',
        'Organizational issues': 'cat_Organization',
        'Not determined': 'cat_NotDetermined'
    }
    for label, new_col in cat_map.items():
        input_df[new_col] = (input_df['category_no'] == label).astype(int)

    # Group by both ev_id and Aircraft_Key, summing the indicator columns
    grouped_df = input_df.groupby(['ev_id', 'Aircraft_Key'], as_index=False)[list(cat_map.values())].sum()

    # Strip whitespace from string columns
    grouped_df = grouped_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Return the cleaned dataframe
    return grouped_df

# Apply cleaning function:
findings_df = findings_cleaner(findings_df)

# Display outputs:
display(findings_df)

,ev_id,Aircraft_Key,cat_Aircraft,cat_Personnel,cat_Environmental,cat_Organization,cat_NotDetermined
0,20231018193265,1,0,1,0,0,0
1,20231018193266,1,0,0,2,0,0
2,20231019193271,1,3,1,0,0,0
3,20231020193275,1,1,1,0,0,1
4,20231023193279,1,0,0,1,0,0
...,...,...,...,...,...,...,...
22874,20231017193256,1,1,1,0,0,0
22875,20231017193257,1,1,1,1,0,0
22876,20231017193258,1,3,1,0,0,0
22877,20231018193262,1,2,2,3,0,0


## Merge Data:

### Findings and Aircraft data:

In [10]:
# Merge aircraft, findings, and flightcrew on 'ev_id' and 'Aircraft_Key':
aircraft_findings_df = pd.merge(
    aircraft_df, 
    findings_df, 
    on=['ev_id', 'Aircraft_Key'], 
    how='outer'
)

def cat_filler(input_df):
    # If Findings are missing, set 'cat_NotDetermined' to 1 and the other 'cat_' features to 0:
    cat_cols = [
        'cat_Aircraft',
        'cat_Personnel',
        'cat_Environmental',
        'cat_Organization',
        'cat_NotDetermined',
        ]

    # Identify rows where all these cat_ columns are missing
    missing_mask = input_df[cat_cols].isna().all(axis=1)

    # Fill in for missing rows
    input_df.loc[missing_mask, cat_cols] = [0, 0, 0, 0, 1]

    return input_df

### Merge in Flight Crew data:

In [11]:
# Now merge in Flight crew data:
merged_all = pd.merge(aircraft_findings_df, flightcrew_df, on=['ev_id', 'Aircraft_Key'], how='outer')

# Merge events on 'ev_id':
partial_merge = pd.merge(events_df, merged_all, on='ev_id', how='left')

# Merge airports on 'ev_nr_apt_id' and 'Loc_Id':
merged_df = pd.merge(partial_merge, airport_df, left_on='ev_nr_apt_id', right_on='Loc_Id', how='left')

# Fill missing incident categories:
merged_df = cat_filler(merged_df)

# Fill remaining missing airport values using 'dprt_apt_id':
dprt_merged = pd.merge(partial_merge, airport_df, left_on='dprt_apt_id', right_on='Loc_Id', how='left', suffixes=('', '_dprt'))
for col in ['Owner_Type', 'Fac_Use', 'Dist_City_', 'Elevation', 'Region_Cod']:
    merged_df[col] = merged_df[col].replace(['', ' '], pd.NA)
    merged_df[col] = merged_df[col].fillna(dprt_merged[col])

# Fill remaining missing airport values using 'dest_apt_id':
dest_merged = pd.merge(partial_merge, airport_df, left_on='dest_apt_id', right_on='Loc_Id', how='left', suffixes=('', '_dest'))
for col in ['Owner_Type', 'Fac_Use', 'Dist_City_', 'Elevation', 'Region_Cod']:
    merged_df[col] = merged_df[col].replace(['', ' '], pd.NA)
    merged_df[col] = merged_df[col].fillna(dest_merged[col])

# Display outputs:
display(merged_df)

,ev_id,ev_date,ev_time,ev_year,ev_month,ev_nr_apt_loc,ev_nr_apt_id,apt_dist,apt_elev,light_cond,sky_cond_ceil,sky_cond_nonceil,sky_ceil_ht,sky_nonceil_ht,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_vel_kts,gust_kts,altimeter,ev_highest_injury,wx_cond_basic,Aircraft_Key,regis_no,ntsb_no,dest_apt_id,dprt_apt_id,damage,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,fixed_retractable,type_last_insp,afm_hrs_last_insp,date_last_insp,afm_hrs,rwy_len,rwy_width,rwy_num,cat_Aircraft,cat_Personnel,cat_Environmental,cat_Organization,cat_NotDetermined,crew_no,crew_age,med_certf,Loc_Id,Owner_Type,Fac_Use,Dist_City_,Elevation,Region_Cod,Land_Area_
0,20080211X00175,2008-01-10,19.12,2008,1,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,32.0,32.0,0.0,NaN,0.0,0.000000,1.0,NaN,1.0,N530NA,DFW08RA039,NaN,NaN,3.0,NaN,HELI,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20080107X00026,2008-01-01,22.50,2008,1,1.0,OQ3,0.0,10.0,2.0,2.0,2.0,0.0,0.0,10.0,134.6,89.6,80.0,10.0,0.0,30.190001,1.0,2.0,1.0,N8037W,SEA08LA057A,OQ3,O69,2.0,2400.0,AIR,0.0,4.0,1.0,1.0,3.0,75.0,2007-04-01,2865.0,2700.0,45.0,7,0.0,1.0,0.0,0.0,0.0,1.0,56.0,3.0,OQ3,0.0,1.0,4.0,16.0,AWP,79.0
2,20080107X00026,2008-01-01,22.50,2008,1,1.0,OQ3,0.0,10.0,2.0,2.0,2.0,0.0,0.0,10.0,134.6,89.6,80.0,10.0,0.0,30.190001,1.0,2.0,2.0,N15EX,SEA08LA057B,O70,OQ3,3.0,1960.0,AIR,0.0,2.0,1.0,1.0,4.0,NaN,2007-01-01,340.0,2700.0,45.0,7,0.0,1.0,0.0,0.0,0.0,1.0,70.0,3.0,OQ3,0.0,1.0,4.0,16.0,AWP,79.0
3,20080109X00036,2008-01-01,22.00,2008,1,1.0,AXH,0.0,68.0,2.0,2.0,2.0,0.0,0.0,10.0,131.0,57.2,360.0,15.0,20.0,30.620001,1.0,2.0,1.0,N579RM,DFW08CA054,AXH,T20,3.0,2850.0,AIR,0.0,2.0,1.0,1.0,3.0,NaN,2007-09-01,516.0,5003.0,100.0,090,1.0,0.0,1.0,0.0,0.0,1.0,54.0,3.0,AXH,0.0,1.0,15.0,68.9,ASW,165.0
4,20080107X00027,2008-01-03,8.42,2008,1,1.0,PWA,0.0,1299.0,6.0,2.0,2.0,0.0,0.0,10.0,32.0,32.0,170.0,16.0,0.0,30.639999,4.0,2.0,1.0,N398J,DFW08LA055,PWA,PHX,1.0,9920.0,AIR,0.0,9.0,1.0,1.0,0.0,NaN,NaT,1871.0,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,1.0,59.0,2.0,PWA,1.0,1.0,7.0,1299.6,ASW,1143.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32023,20250225199761,2025-02-24,13.80,2025,2,NaN,NaN,NaN,NaN,2.0,0.0,3.0,0.0,3760.0,NaN,NaN,NaN,320.0,5.0,0.0,NaN,2.0,2.0,1.0,N942AT,DCA25LA139,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,2.0,47.8,ASO,NaN
32024,20250226199769,2025-02-26,13.50,2025,2,0.0,NaN,0.0,NaN,2.0,4.0,0.0,1100.0,0.0,7.0,98.6,93.2,0.0,NaN,0.0,29.799999,3.0,1.0,1.0,N43DN,ERA25LA130,NaN,MTN,1.0,NaN,AIR,0.0,4.0,1.0,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32025,20250227199772,2025-02-26,20.63,2025,2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,1.0,2.0,1.0,N955NN,OPS25LA025A,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,21.5,ASO,NaN
32026,20250227199772,2025-02-26,20.63,2025,2,NaN,NaN,NaN,NaN,2.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,1.0,2.0,2.0,N322RC,OPS25LA025B,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## (Surya) Fill Missing 'crew_no' for 'acft_category'=='AIR'

In [12]:
# Impute stuff:
merged_df







# Get count of missing values:
print(merged_df.isna().sum()[merged_df.isna().sum() > 0])

ev_time                370
ev_nr_apt_loc         4273
ev_nr_apt_id         11894
apt_dist               139
apt_elev             13742
light_cond            5039
vis_sm                6345
wx_temp                139
wx_dew_pt              139
wind_dir_deg            33
wind_vel_kts         10752
gust_kts                33
altimeter              139
wx_cond_basic         5023
Aircraft_Key            68
regis_no               498
ntsb_no                453
dest_apt_id          11155
dprt_apt_id           8692
damage                 453
cert_max_gr_wt        9866
acft_category          846
homebuilt              453
total_seats           6699
num_eng               4630
fixed_retractable      453
type_last_insp        7980
afm_hrs_last_insp    23273
date_last_insp       10561
afm_hrs              10961
rwy_len              17549
rwy_width            17614
rwy_num              17222
crew_no               7631
crew_age              7631
med_certf             7631
Loc_Id               14090
O

# Cleaning Merged Data:

### Fill 'Unknown' and 'Not Reported':

In [13]:
# Fill N/A values with:
merged_df['light_cond'] = merged_df['light_cond'].fillna(0)
merged_df['type_last_insp'] = merged_df['type_last_insp'].fillna(0)

features = ['light_cond', 'sky_cond_ceil', 'sky_cond_nonceil']
target = 'wx_cond_basic'

# Separate rows with known wx_cond_basic (train) vs. missing (test)
known_mask = merged_df[target].notna()
missing_mask = merged_df[target].isna()

# Fill with KNN Classifier:
train_df = merged_df[known_mask].copy()
test_df = merged_df[missing_mask].copy()

# Prepare train data
X_train = train_df[features]
y_train = train_df[target]

# Prepare test data
X_test = test_df[features]

# Build and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict missing values
y_pred = knn.predict(X_test)

# Assign predictions back
merged_df.loc[missing_mask, target] = y_pred

### Propagate Elevation and 'afm_hrs_last_insp':

In [14]:
# Combine missing elevation data:
merged_df['apt_elev'] = merged_df['apt_elev'].fillna(merged_df['Elevation'])
merged_df['Elevation'] = merged_df['Elevation'].fillna(merged_df['apt_elev'])

# Calculate hours since last inspection:
time_diff_hours = (merged_df['ev_date'] - merged_df['date_last_insp']).dt.days * 24

# Fill missing 'afm_hrs_last_insp':
merged_df['afm_hrs_last_insp'] = merged_df['afm_hrs_last_insp'].fillna(time_diff_hours)

# Explicitly fill any remaining NaNs with 0 (or another value):
merged_df['afm_hrs_last_insp'] = merged_df['afm_hrs_last_insp'].fillna(0)

# Convert to integer:
merged_df['afm_hrs_last_insp'] = merged_df['afm_hrs_last_insp'].round(0).astype('Int64')

# Drop excess columns:
merged_df = merged_df.drop(columns=['ev_id', 'Aircraft_Key', 'ev_nr_apt_id', 'Loc_Id', 
                                    'ntsb_no', 'regis_no', 'dest_apt_id', 'dprt_apt_id',
                                    'Land_Area_'
                                    ])

# Check for missing values:
print(merged_df.isna().sum()[merged_df.isna().sum() > 0])

ev_time                370
ev_nr_apt_loc         4273
apt_dist               139
apt_elev              2506
vis_sm                6345
wx_temp                139
wx_dew_pt              139
wind_dir_deg            33
wind_vel_kts         10752
gust_kts                33
altimeter              139
damage                 453
cert_max_gr_wt        9866
acft_category          846
homebuilt              453
total_seats           6699
num_eng               4630
fixed_retractable      453
date_last_insp       10561
afm_hrs              10961
rwy_len              17549
rwy_width            17614
rwy_num              17222
crew_no               7631
crew_age              7631
med_certf             7631
Owner_Type            2764
Fac_Use               2764
Dist_City_            2764
Elevation             2506
Region_Cod            2839
dtype: int64


## Impute based on acft_category:

In [15]:
print(merged_df['acft_category'].value_counts())

acft_category
AIR     26660
HELI     3342
GLI       440
BALL      215
WSFT      193
GYRO      168
PPAR      108
ULTR       29
UNK        20
PLFT        5
RCKT        1
BLIM        1
Name: count, dtype: int64


In [16]:
# Identify rows where acft_category is NOT 'AIR' or 'HELI'
non_acft_mask = ~merged_df['acft_category'].isin(['AIR', 'HELI'])
non_acft_df = merged_df.loc[non_acft_mask].copy()

# Impute 'crew_age' and 'crew_no' by the mode per acft_category
for category in non_acft_df['acft_category'].unique():
    cat_mask = (non_acft_df['acft_category'] == category)
    # For each column that needs mode-based imputation
    for col in ['crew_age', 'crew_no']:
        # Calculate the mode (most frequent value) for that category
        mode_val = non_acft_df.loc[cat_mask, col].mode(dropna=True)
        if not mode_val.empty:
            # Fill missing values with this mode
            non_acft_df.loc[cat_mask, col] = non_acft_df.loc[cat_mask, col].fillna(mode_val.iloc[0])

# If not 'AIR' or 'HELI', set 'crew_no' to 1:
non_acft_df['crew_no'] = non_acft_df['crew_no'].fillna(1)

# Fill N/A with 0 for 'med_certf', 'rwy_len', 'rwy_num', 'rwy_width'
fill_zero_cols = ['med_certf', 'rwy_len', 'rwy_num', 'rwy_width']
non_acft_df[fill_zero_cols] = non_acft_df[fill_zero_cols].fillna(0)

# Update the original DataFrame with these changes
merged_df.loc[non_acft_mask] = non_acft_df

# Check how many missing values remain:
print('Missing values after imputation:')
print(merged_df[['med_certf', 'rwy_len', 'rwy_num', 'rwy_width', 'crew_age', 'crew_no',]].isna().sum())

Missing values after imputation:
med_certf     7062
rwy_len      16001
rwy_num      15691
rwy_width    16061
crew_age      7477
crew_no       7062
dtype: int64


In [17]:
# Convert 'acft_category' to numeric
label_enc = LabelEncoder()
# We handle missing or unknown categories by filling them with a placeholder before encoding
merged_df['acft_category'] = merged_df['acft_category'].fillna('UNK')

# Fit the encoder on all observed categories
label_enc.fit(merged_df['acft_category'])

# Transform the string categories into numeric codes
merged_df['acft_category_encoded'] = label_enc.transform(merged_df['acft_category'])

# Prepare columns for KNN imputation of 'cert_max_gr_wt'
k_cols = ['acft_category_encoded', 'cert_max_gr_wt']
temp_df = merged_df[k_cols].copy()

# Convert to float for KNNImputer
temp_df = temp_df.astype(float, errors='ignore')

# Create the imputer
imputer = KNNImputer(n_neighbors=5)

# Fit/transform to get imputed data
imputed_data = imputer.fit_transform(temp_df)

# Assign the imputed values back to 'cert_max_gr_wt'
merged_df['cert_max_gr_wt'] = imputed_data[:, 1]

# (Optional) drop the encoded helper column if you no longer need it
merged_df.drop(columns=['acft_category_encoded'], inplace=True)

# KNN imputation for ev_nr_apt_loc based on apt_dist
k_cols2 = ['ev_nr_apt_loc', 'apt_dist']
temp_df2 = merged_df[k_cols2].copy()

# Ensure numeric types for KNNImputer
temp_df2 = temp_df2.astype(float, errors='ignore')

imputer2 = KNNImputer(n_neighbors=5)
imputed_data2 = imputer2.fit_transform(temp_df2)

# Fill missing maintenance hours:
merged_df['afm_hrs'] = merged_df['afm_hrs'].fillna(0)

# Put the imputed ev_nr_apt_loc back into merged_df
merged_df['ev_nr_apt_loc'] = imputed_data2[:, 0]

# Continue Imputing:

In [18]:
# Drop excess columns:
purged_df = merged_df.drop(columns=['Elevation', 'ev_date', 'date_last_insp','wind_vel_kts', 'rwy_len', 'rwy_width', 'rwy_num', 'Dist_City_', 'crew_age',])

# Drop rows missing 'wind_dir_deg' (very few missing):
purged_df = purged_df.dropna(subset=['wind_dir_deg', 'apt_dist', 'ev_time', 'homebuilt',])

# Fill missing values with 0:
purged_df['afm_hrs'] = purged_df['afm_hrs'].fillna(0)
purged_df['total_seats'] = purged_df['total_seats'].fillna(0)
purged_df['med_certf'] = purged_df['med_certf'].fillna(0)

# Fill with mode, based on acft_category:
imputer = SimpleImputer(strategy='most_frequent')
for category, subdf in purged_df.groupby('acft_category'):
    purged_df.loc[subdf.index, 'num_eng'] = imputer.fit_transform(subdf[['num_eng']])

### Impute Visibility:

In [19]:
# Columns to use for imputing 'vis_sm'
impute_features = [
    'vis_sm',
    'ev_time',
    'light_cond',
    'sky_cond_ceil',
    'sky_cond_nonceil',
    'sky_ceil_ht',
    'sky_nonceil_ht',
    'wx_cond_basic',
    'wx_dew_pt',
    'wx_temp',
    ]

# Check which columns are actually present in purged_df
available_cols = [col for col in impute_features if col in purged_df.columns]
missing_cols = set(impute_features) - set(available_cols)
if missing_cols:
    print(f'Warning: these columns are missing in purged_df: {missing_cols}:')

# Convert these columns to categorical codes to ensure KNN can handle them
categorical_cols = ['ev_time', 'light_cond', 'sky_cond_ceil', 'sky_cond_nonceil', 'wx_cond_basic']
for col in categorical_cols:
    if col in available_cols:
        purged_df[col] = purged_df[col].astype('category').cat.codes

# Prepare the data for imputation
X = purged_df[available_cols].copy()

# Perform KNN imputation
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(X)

# Put the imputed data back into purged_df
purged_df[available_cols] = X_imputed

### Fill missing Helicopter Crew Numbers:

In [20]:
# Identify missing crew_no values for helicopters:
heli_mask = (purged_df['acft_category'] == 'HELI') & (purged_df['crew_no'].isna())
missing_indices = purged_df[heli_mask].index
n_missing = len(missing_indices)

# We'll fill 1/3 with 2, and the rest with 1 as well:
one_third_count = n_missing // 3
chosen_indices = np.random.choice(missing_indices, size=one_third_count, replace=False)

# Fill that subset with 2:
purged_df.loc[chosen_indices, 'crew_no'] = 2

# Fill the rest with 1:
remaining_indices = missing_indices.difference(chosen_indices)
purged_df.loc[remaining_indices, 'crew_no'] = 1

# Output N/A amount:
print(f'Total records: {len(purged_df):,}\n')
print(f'Total records (w/dropna()): {len(purged_df.dropna()):,}\n')
print(purged_df.isna().sum()[purged_df.isna().sum() > 0])

Total records: 31,085

Total records (w/dropna()): 23,734

apt_elev      2194
crew_no       5942
Owner_Type    2432
Fac_Use       2432
Region_Cod    2504
dtype: int64


### Clean up Target Feature names:

In [21]:
display(purged_df['damage'].value_counts())
display(purged_df['ev_highest_injury'].value_counts())

damage
3.0    24620
4.0     3431
1.0     1732
2.0     1124
0.0      178
Name: count, dtype: int64

ev_highest_injury
1.0    17135
4.0     6373
2.0     4173
3.0     3404
Name: count, dtype: int64

# Final Null Drop:

In [22]:
# Identify how many features are null within each event:
purged_df['null_feats'] = purged_df.isna().sum(axis=1)

# Sort by number of nulls:
purged_df = purged_df.sort_values(by='null_feats', ascending=False)

# Move 'null_feats' to front of DataFrame:
purged_df = purged_df[['null_feats'] + [col for col in purged_df.columns if col != 'null_feats']]

# Drop observations with mostly null features:
purged_df = purged_df.loc[purged_df['null_feats'] < 30]

# Re-sort data:
purged_df = purged_df.sort_index().reset_index(drop=True)

# Display output:
display(purged_df)

,null_feats,ev_time,ev_year,ev_month,ev_nr_apt_loc,apt_dist,apt_elev,light_cond,sky_cond_ceil,sky_cond_nonceil,sky_ceil_ht,sky_nonceil_ht,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,gust_kts,altimeter,ev_highest_injury,wx_cond_basic,damage,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,fixed_retractable,type_last_insp,afm_hrs_last_insp,afm_hrs,cat_Aircraft,cat_Personnel,cat_Environmental,cat_Organization,cat_NotDetermined,crew_no,med_certf,Owner_Type,Fac_Use,Region_Cod
0,4,1101.0,2008,1,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,10.0,32.0,32.0,0.0,0.0,0.000000,1.0,1.0,3.0,2970.2,HELI,0.0,0.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN
1,0,1304.0,2008,1,1.0,0.0,10.0,2.0,2.0,1.0,0.0,0.0,10.0,134.6,89.6,80.0,0.0,30.190001,1.0,1.0,2.0,2400.0,AIR,0.0,4.0,1.0,1.0,3.0,75,2865.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,AWP
2,0,1304.0,2008,1,1.0,0.0,10.0,2.0,2.0,1.0,0.0,0.0,10.0,134.6,89.6,80.0,0.0,30.190001,1.0,1.0,3.0,1960.0,AIR,0.0,2.0,1.0,1.0,4.0,8760,340.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,AWP
3,0,1274.0,2008,1,1.0,0.0,68.0,2.0,2.0,1.0,0.0,0.0,10.0,131.0,57.2,360.0,20.0,30.620001,1.0,1.0,3.0,2850.0,AIR,0.0,2.0,1.0,1.0,3.0,2928,516.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,1.0,ASW
4,0,475.0,2008,1,1.0,0.0,1299.0,6.0,2.0,1.0,0.0,0.0,10.0,32.0,32.0,170.0,0.0,30.639999,4.0,1.0,1.0,9920.0,AIR,0.0,9.0,1.0,1.0,0.0,0,1871.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0,ASW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31080,0,1064.0,2025,2,1.0,0.0,1494.0,0.0,2.0,1.0,0.0,0.0,10.0,113.0,93.2,320.0,0.0,30.240000,1.0,1.0,3.0,2970.2,HELI,0.0,0.0,6.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,ANM
31081,1,1214.0,2025,2,1.0,0.0,2645.0,0.0,3.0,0.0,300.0,0.0,9.0,86.0,86.0,90.0,0.0,30.180000,1.0,1.0,3.0,3877.8,AIR,0.0,4.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,1.0,ANM
31082,1,959.0,2025,2,1.0,1.0,1618.0,0.0,2.0,1.0,0.0,0.0,10.0,113.0,82.4,260.0,23.0,29.400000,1.0,1.0,3.0,3877.8,AIR,0.0,4.0,1.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,1.0,AGL
31083,1,1094.0,2025,2,1.0,0.0,63.0,2.0,2.0,1.0,0.0,0.0,10.0,161.6,122.0,0.0,0.0,30.160000,2.0,1.0,3.0,1430.0,AIR,0.0,2.0,1.0,1.0,3.0,0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,1.0,ASO


In [23]:
display(purged_df['damage'].value_counts())
display(purged_df['ev_highest_injury'].value_counts())

damage
3.0    24620
4.0     3431
1.0     1732
2.0     1124
0.0      178
Name: count, dtype: int64

ev_highest_injury
1.0    17135
4.0     6373
2.0     4173
3.0     3404
Name: count, dtype: int64

# Save Final Data:

In [24]:
# Rename features to 'Damage' and 'Injury':
rename_list = ['damage', 'ev_highest_injury']
clean_df = purged_df.rename(columns=dict(zip(rename_list, ['Damage', 'Injury'])))

# Define mappings (example mappings – adjust as needed):
damage_map = {
    1: 1, # 'None/Minor'
    2: 1, # 'None/Minor'
    3: 2, # 'Substantial'
    4: 3, # 'Destroyed'
    0: 1, # 'None/Minor'
    }

injury_map = {
    1: 1, # 'None/Minor'
    2: 1, # 'None/Minor'
    3: 2, # 'Serious'
    4: 3, # 'Fatal'
    0: 1, # 'Unknown'
    }

# Map integer values to strings:
clean_df['Damage'] = clean_df['Damage'].map(damage_map)
clean_df['Injury'] = clean_df['Injury'].map(injury_map)

# Drop final missing N/A values:
clean_df = clean_df.dropna(ignore_index=True)

# Save results:
clean_df.to_csv('../Datasets/aviation_df.csv', index=False)
display(clean_df['Damage'].value_counts())
display(clean_df['Injury'].value_counts())

Damage
2    20234
3     2348
1     1152
Name: count, dtype: int64

Injury
1    16428
3     4556
2     2750
Name: count, dtype: int64

### Final Class Count:

In [25]:
# Copy DataFrame:
output_df = clean_df.copy()

# Define mappings (example mappings – adjust as needed):
damage_remap = {
    1: 'None/Minor',
    2: 'Substantial',
    3: 'Destroyed',
    }

injury_remap = {
    1: 'None/Minor',
    2: 'Serious',
    3: 'Fatal',
    }

# Map integer values to strings:
output_df['Damage'] = output_df['Damage'].map(damage_remap)
output_df['Injury'] = output_df['Injury'].map(injury_remap)

# Save results:
output_df.to_csv('./aviation_df.csv', index=False)
display(output_df['Damage'].value_counts())
display(output_df['Injury'].value_counts())

Damage
Substantial    20234
Destroyed       2348
None/Minor      1152
Name: count, dtype: int64

Injury
None/Minor    16428
Fatal          4556
Serious        2750
Name: count, dtype: int64